In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder


In [5]:
sns.set(font_scale=1.5)

In [6]:
df = pd.read_csv( "resd_clean.csv" )
df.head()

,index,sub_area,n_bhk,surface,price,company_name,township,club_house,school,hospital,mall,park,pool,gym
0,1,bavdhan,1.0,492.0,3.9,shapoorji paloonji,vanaha,yes,yes,yes,yes,yes,yes,yes
1,2,bavdhan,2.0,774.0,6.5,shapoorji paloonji,vanaha,yes,yes,yes,yes,yes,yes,yes
2,3,bavdhan,3.0,889.0,7.4,shapoorji paloonji,vanaha,yes,yes,yes,yes,yes,yes,yes
3,4,bavdhan,3.0,1018.0,8.9,shapoorji paloonji,vanaha,yes,yes,yes,yes,yes,yes,yes
4,5,mahalunge,2.0,743.0,7.4,godrej properties,godrej hills retreat,yes,yes,yes,yes,yes,yes,yes


In [8]:
df = df.drop(columns=["index", "company_name", "township"]).drop_duplicates()
df.head()

,sub_area,n_bhk,surface,price,club_house,school,hospital,mall,park,pool,gym
0,bavdhan,1.0,492.0,3.9,yes,yes,yes,yes,yes,yes,yes
1,bavdhan,2.0,774.0,6.5,yes,yes,yes,yes,yes,yes,yes
2,bavdhan,3.0,889.0,7.4,yes,yes,yes,yes,yes,yes,yes
3,bavdhan,3.0,1018.0,8.9,yes,yes,yes,yes,yes,yes,yes
4,mahalunge,2.0,743.0,7.4,yes,yes,yes,yes,yes,yes,yes


In [10]:
binary_cols = df.iloc[:, 4:].columns.to_list()

laben = LabelEncoder()
for col in binary_cols:
    df[col] = laben.fit_transform(df[col])
    
df.tail(10)


,sub_area,n_bhk,surface,price,club_house,school,hospital,mall,park,pool,gym
185,bavdhan,2.0,665.0,6.499,1,0,0,0,1,0,0
186,talegoan,1.0,280.0,1.319,1,0,0,0,1,1,1
187,talegoan,2.0,385.0,1.729,1,0,0,0,1,0,1
188,baner,2.0,730.0,8.700,1,0,0,0,1,1,1
189,baner,3.0,1135.0,12.900,1,0,0,0,1,1,1
190,baner,3.0,1350.0,16.000,1,0,0,0,1,1,1
191,baner,4.0,1600.0,23.000,1,0,0,0,1,1,1
192,baner,2.0,789.0,9.500,0,0,0,0,1,1,1
193,baner,3.0,1107.0,13.500,0,0,0,0,1,1,1
194,baner,2.0,776.0,9.068,1,0,0,0,1,1,1


In [11]:
df_sa_count = df.groupby("sub_area")["price"].count().reset_index()\
                .rename(columns={"price":"count"})\
                .sort_values("count", ascending=False)\
                .reset_index(drop=True)

In [12]:
df_sa_count["sa_contribution"] = df_sa_count["count"]/len(df)

In [14]:


sa_sel_col = df_sa_count.loc[df_sa_count["count"]>7, "sub_area"].to_list()
df["sub_area"] = df["sub_area"].where(df["sub_area"].isin(sa_sel_col), "other")
df["sub_area"].unique(), len(df["sub_area"].unique())
hoten = OneHotEncoder(sparse=False)
X_dummy = hoten.fit_transform( df[["sub_area"]] )
X_dummy.shape, X_dummy, X_dummy.dtype


C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


((195, 12),
 array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 dtype('float64'))

In [17]:
X_dummy = X_dummy.astype("int64")

In [18]:
sa_cols_name = ["sa"+str(i+1) for i in range(X_dummy.shape[1])]
df.loc[:,sa_cols_name] = X_dummy
df.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_20764\2821117971.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,sa_cols_name] = X_dummy


,sub_area,n_bhk,surface,price,club_house,school,hospital,mall,park,pool,...,sa3,sa4,sa5,sa6,sa7,sa8,sa9,sa10,sa11,sa12
0,bavdhan,1.0,492.0,3.9,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,bavdhan,2.0,774.0,6.5,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,bavdhan,3.0,889.0,7.4,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,bavdhan,3.0,1018.0,8.9,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,mahalunge,2.0,743.0,7.4,1,1,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0


In [19]:
df[["sub_area"]+sa_cols_name].drop_duplicates()\
            .sort_values("sub_area").reset_index(drop=True)

,sub_area,sa1,sa2,sa3,sa4,sa5,sa6,sa7,sa8,sa9,sa10,sa11,sa12
0,baner,1,0,0,0,0,0,0,0,0,0,0,0
1,bavdhan,0,1,0,0,0,0,0,0,0,0,0,0
2,bt kawade rd,0,0,1,0,0,0,0,0,0,0,0,0
3,hadapsar,0,0,0,1,0,0,0,0,0,0,0,0
4,handewadi,0,0,0,0,1,0,0,0,0,0,0,0
5,hinjewadi,0,0,0,0,0,1,0,0,0,0,0,0
6,kharadi,0,0,0,0,0,0,1,0,0,0,0,0
7,mahalunge,0,0,0,0,0,0,0,1,0,0,0,0
8,nibm,0,0,0,0,0,0,0,0,1,0,0,0
9,other,0,0,0,0,0,0,0,0,0,1,0,0


In [20]:
data = df.select_dtypes(exclude="object")
float_cols = data.select_dtypes( include="float" ).columns.to_list()

In [22]:
corr_cols_list.remove("price")
corr_cols_list.extend(sa_cols_name)
corr_list = [] # to keep the correlations with price
for col in corr_cols_list:
    corr_list.append( round(data["price"].corr(data[col]),2) )    

corr_list

[0.7,
 0.88,
 0.02,
 -0.04,
 -0.04,
 -0.04,
 0.08,
 0.29,
 0.06,
 0.33,
 -0.11,
 0.08,
 -0.04,
 -0.17,
 0.0,
 -0.11,
 -0.05,
 0.23,
 -0.03,
 -0.0,
 -0.07]

In [23]:
df_corr = pd.DataFrame( data=zip(corr_cols_list, corr_list), 
                 columns=["col_name", "corr"] )\
            .sort_values("corr", ascending=False)\
            .reset_index(drop=True)

In [25]:
features = df_corr.loc[abs(df_corr["corr"])>.1, "col_name"].to_list() 
features, len(features)


(['surface', 'n_bhk', 'sa1', 'pool', 'sa9', 'sa7', 'sa2', 'sa5'], 8)

In [26]:
sa_feature_list = [sa for sa in features if "sa" in sa]
lst = []
for col in sa_feature_list:
    sa_triger = df[col]==1
    sa = df.loc[sa_triger, "sub_area"].to_list()[0]
    x = df.loc[sa_triger, "price"]
    lst.append( (sa, np.mean(x), df[col].sum()) )

lst

[('baner', 16.038999999999998, 12),
 ('nibm', 15.1, 8),
 ('kharadi', 6.6310625000000005, 16),
 ('bavdhan', 5.804400000000001, 10),
 ('handewadi', 4.568818181818182, 11)]

In [30]:



sel_data = data[features+["price"]].copy()
sel_data.head()
sc = StandardScaler(with_std=True, with_mean=True)
sel_data["surface"] = sc.fit_transform(sel_data[["surface"]])
sel_data


,surface,n_bhk,sa1,pool,sa9,sa7,sa2,sa5,price
0,-1.089917,1.0,0,1,0,0,1,0,3.900
1,-0.331617,2.0,0,1,0,0,1,0,6.500
2,-0.022381,3.0,0,1,0,0,1,0,7.400
3,0.324501,3.0,0,1,0,0,1,0,8.900
4,-0.414976,2.0,0,1,0,0,0,0,7.400
...,...,...,...,...,...,...,...,...,...
190,1.217251,3.0,1,1,0,0,0,0,16.000
191,1.889503,4.0,1,1,0,0,0,0,23.000
192,-0.291282,2.0,1,1,0,0,0,0,9.500
193,0.563823,3.0,1,1,0,0,0,0,13.500


In [32]:
sel_data.to_csv("resd_features.csv", index=False)

In [35]:

data = pd.read_csv("resd_features.csv")
data.shape


(195, 9)

In [37]:
data.describe()

,surface,n_bhk,sa1,pool,sa9,sa7,sa2,sa5,price
count,1.950000e+02,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000
mean,1.275333e-16,2.379487,0.061538,0.630769,0.041026,0.082051,0.051282,0.056410,8.669949
std,1.002574e+00,0.846210,0.240934,0.483839,0.198860,0.275149,0.221140,0.231306,5.808464
min,-1.659986e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.319000
25%,-6.112734e-01,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.100000
50%,-2.724585e-01,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.000000
75%,3.957598e-01,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,9.700000
max,4.167092e+00,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,46.000000


In [ ]:
# sa1: baner, sa9: nibm, sa7: kharadi, sa2: bavdhan, sa5: handewadi

In [38]:
data = data.sort_values("surface").reset_index(drop=True)

In [39]:
X = data.iloc[:, :-1]
y = data["price"]

In [40]:
rs = 118
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.3, random_state=rs) 

In [42]:
model_reg= final_model('random',X,y,rs,X_train, X_test, y_train, y_test)
print("Regression Model Executed")

NameError: name 'final_model' is not defined

In [43]:
rs=13
y_scaled = y_scaled_val(y)
X_train, X_test, y_train, y_test = train_test_split(X.values, y_scaled.values, #train test split
                                                    test_size=.3, 
                                                    random_state=rs)

mlp_tensorflow_model= mlp_tf_model(X_train, X_test, y_train, y_test)
mlp_tensorflow_model.summary()

NameError: name 'y_scaled_val' is not defined

In [44]:
plot_model_history(mlp_tensorflow_model.history)  ## plot the training history
plotResidueMLP(mlp_tensorflow_model, X, y_scaled, rs=rs)  #plot residual 
plot_real_pred(mlp_tensorflow_model, X, y_scaled, rs) # plot predictions
print("MLP Model Executed")

NameError: name 'plot_model_history' is not defined